In [1]:
import pennylane as qml
from pennylane import numpy as np

from qbmqsp.hamiltonian import Hamiltonian
from qbmqsp.block_encode import BlockEncode

In [2]:
def test_block_encoding(scheme):
    h = ["ZZII", "IZZI", "IIZZ", "ZIIZ",
         "XIII", "IXII", "IIXI", "IIIX",
         "YIII", "IYII", "IIYI", "IIIY"]
    n_params = len(h)
    np.random.seed(seed=1)
    θ = np.random.random(n_params) / n_params

    block_encode = BlockEncode(h, θ, scheme)
    n_enc = block_encode.n_qubits('enc')
    n_sys = block_encode.n_qubits('sys')
    wires = list(range(n_enc + n_sys))
    enc_wires = wires[: n_enc]
    sys_wires = wires[n_enc : n_enc+n_sys]
    
    dev = qml.device('default.qubit', wires=n_enc+n_sys)
    @qml.qnode(dev)
    def block_encoding():
        block_encode.circuit(enc_wires, sys_wires)
        return qml.state()
    out = block_encoding().reshape(2**n_enc, 2**n_sys)[0, :] # = (<0| x I) block_encode |00>
    
    H = Hamiltonian(h, θ)
    ideal = H.assemble()[:, 0] / H.θ_norm()

    error = np.sum(np.abs(out - ideal))
    return error

In [3]:
test_block_encoding('general')

tensor(0., requires_grad=True)

In [4]:
test_block_encoding('lcu')

tensor(2.53798601e-16, requires_grad=True)